<a href="https://colab.research.google.com/github/Seiris21/2022_snowpack_capstone/blob/main/notebooks/data_ingestion/Sentinel_1_and_2_Data_Prep_Ingestion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!sudo apt-get update -y
!sudo apt-get upgrade -y
!pip3 install numpy==1.21
# Install GDAL and Geopandas
!sudo apt-get install libgdal-dev -y
!sudo apt install gdal-bin python-gdal python3-gdal --quiet -y
!sudo apt install python3-rtree --quiet -y
!pip3 install git+git://github.com/geopandas/geopandas.git --quiet

!pip3 install -U tornado

In [ ]:
!python3 --version

In [ ]:
%%capture
%pip install geemap

In [ ]:
%%capture
%pip install "dask[complete]"
%pip install "dask[complete]" --upgrade

In [ ]:
%%capture
%pip install pystac_client planetary_computer rasterio xarray-spatial

In [ ]:
%%capture
!pip3 install matplotlib datetime pystac_client planetary_computer xarray datashader xarray-spatial 

In [ ]:
%%capture
!pip3 install rasterio geotiff geopy shapely imagecodecs

In [ ]:
%%capture
!pip3 install sklearn

In [ ]:
%%capture
!pip3 install requests earthengine-api datetime
!pip3 install earthengine-api --upgrade
!pip3 install httplib2==0.15.0

In [ ]:
%%capture
!pip3 install rioxarray

In [1]:
import os
from os import listdir
from os.path import isfile, join

import imagecodecs
from geotiff import GeoTiff
import geopy
import geopy.distance as distance
from shapely.geometry import Polygon

import re
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None
import geopandas as gpd
import matplotlib as mpl
import matplotlib.pyplot as plt

from datetime import datetime, timedelta

from pystac_client import Client
import planetary_computer
import xarray
import dask.dataframe as dd
import xrspatial
from datashader.transfer_functions import shade, stack
from datashader.colors import Elevation
from datashader.utils import export_image

from sklearn.neighbors import BallTree

import shutil
import requests
import ee

import time 
import signal

import rioxarray
import rasterio
import rasterio.features
import shapely
from shapely import wkt


class TimeoutException(Exception):   # Custom exception class
    pass

def timeout_handler(signum, frame):   # Custom signal handler
    raise TimeoutException

/home/ubuntu/.local/lib/python3.8/site-packages/numba/core/errors.py:175: UserWarning: Insufficiently recent colorama version found. Numba requires colorama >= 0.3.9
  warnings.warn(msg)


## Loading Packages and docking drive

In [ ]:
#%%capture
!apt-get update
# Install GDAL and Geopandas
!apt-get install libgdal-dev 
!apt install gdal-bin python-gdal python3-gdal --quiet
!apt install python3-rtree --quiet
!pip install git+git://github.com/geopandas/geopandas.git --quiet

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')
# One may also need to add a shortcut to the shared drive like in 
# https://stackoverflow.com/questions/54351852/accessing-shared-with-me-with-colab/54352105#54352105

In [2]:
import numpy as np
import pandas as pd
import geopandas as gpd

import os
import re
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
#from google.colab import files

## Pull in and stage all  
From SnowCast Showdown Data Wrangling.ipynb

In [3]:
df = pd.read_csv("~/SnowData/traindf_allregions.csv")
print(df.head())

   Unnamed: 0                               cell_id        date   SWE  \
0           0  018cf1a1-f945-4097-9c47-0c4690538bb5  2016-01-05  16.4   
1           1  01be2cc7-ef77-4e4d-80ed-c4f8139162c3  2016-01-05  21.1   
2           2  04b7603c-26c6-4004-a8bc-58b2b60e810e  2016-01-05   3.5   
3           3  147d5eb4-e574-47e4-994a-8a2908c06050  2016-01-05  11.7   
4           4  174e3100-c30e-46a4-ac7c-30cd521fc390  2016-01-05  12.9   

    region                                           geometry  \
0  sierras  POLYGON ((-120.614398422312 39.67242331310127,...   
1  sierras  POLYGON ((-119.6082853040981 38.27574773521515...   
2  sierras  POLYGON ((-118.3416607534896 36.56361241293132...   
3  sierras  POLYGON ((-120.8749098547067 39.78296502318875...   
4  sierras  POLYGON ((-119.5454032342098 37.63117397089415...   

   mean_inversed_swe  mean_local_swe  median_local_swe  max_local_swe  ...  \
0          17.255057          10.348             10.80          23.06  ...   
1          18.

In [ ]:
df.dtypes

Transform geometry to polygon. 

In [4]:
df['geometry']=gpd.GeoSeries.from_wkt(df['geometry'])

In [5]:
gdf = gpd.GeoDataFrame(df, geometry='geometry')

print(gdf.head())

   Unnamed: 0                               cell_id        date   SWE  \
0           0  018cf1a1-f945-4097-9c47-0c4690538bb5  2016-01-05  16.4   
1           1  01be2cc7-ef77-4e4d-80ed-c4f8139162c3  2016-01-05  21.1   
2           2  04b7603c-26c6-4004-a8bc-58b2b60e810e  2016-01-05   3.5   
3           3  147d5eb4-e574-47e4-994a-8a2908c06050  2016-01-05  11.7   
4           4  174e3100-c30e-46a4-ac7c-30cd521fc390  2016-01-05  12.9   

    region                                           geometry  \
0  sierras  POLYGON ((-120.61440 39.67242, -120.61440 39.6...   
1  sierras  POLYGON ((-119.60829 38.27575, -119.60829 38.2...   
2  sierras  POLYGON ((-118.34166 36.56361, -118.34166 36.5...   
3  sierras  POLYGON ((-120.87491 39.78297, -120.87491 39.7...   
4  sierras  POLYGON ((-119.54540 37.63117, -119.54540 37.6...   

   mean_inversed_swe  mean_local_swe  median_local_swe  max_local_swe  ...  \
0          17.255057          10.348             10.80          23.06  ...   
1          18.

In [ ]:
gdf.dtypes

In [ ]:
gdf

In [6]:
from datetime import datetime, timedelta
traindf = gdf.loc[pd.to_datetime(gdf.date) >= datetime.strptime("2016-01-01", "%Y-%m-%d")].reset_index(drop = True)

In [ ]:
traindf.head()

In [7]:
import shutil
import os
import requests
import ee
from datetime import datetime, timedelta

traindf["date"] = pd.to_datetime(traindf.date)


#I am creating a string version of the date to use as a filename
traindf["datestring"] = traindf.date.map(lambda d: str(d.year)+d.strftime('%j'))


In [14]:
traindf = traindf.drop('Unnamed: 0', axis = 1).loc[traindf.cell_id.str.contains("ASO")].reset_index(drop = True)

## Sentinel 1 Main Function



In [15]:

def pull_Sentinel(traindf, overwrite = False, names_only = False ):
  ##Main loop that iterates over areas and stores images in file
  ##For test image of one AOI see below
  filelocations = []
  x= 0


  for i in range(len(traindf.SWE)):
    #create a name for the image
    pict_name = traindf.cell_id[i] + '_sentinel1_poly' + traindf.datestring[i] + '.jpg'

    #create the whole filename with path to the correct folder
    filename = os.path.join('/home/ubuntu/SnowData/Sen1_Data_poly/', pict_name)

    if names_only:
      filelocations.append(filename)
      x += 1
      if x % 100 == 0:
        print(f'{x} files already exist')

    elif os.path.exists(filename) and not overwrite:
      filelocations.append(filename)
      x += 1
      if x % 100 == 0:
        print(f'{x} files already exist')

    else:

      #define area of interest by coordinates
      aoi = ee.Geometry.Polygon(list(traindf.geometry[i].exterior.coords))
      #print(aoi)
      start_date = traindf.date[i] - timedelta(days = 80)
      end_date = traindf.date[i] + timedelta(days = 1)

      year = start_date.year
      #print(year)

      if year >=2016:
        x+=1
        try:
          #print("calculating")
          # Sentinel-1 image filtered on date range and on aoi
          se2 = ee.ImageCollection('COPERNICUS/S1_GRD')\
          .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))\
          .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))\
          .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'HV').Not())\
          .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'HH').Not())\
          .filterDate(start_date, end_date)\
          .filterBounds(aoi)\
          .sort('system:time_start', False)\
          .first()

          #Create a ratio band of VV/VH

          VVVH = (se2.select('VV').divide(se2.select('VH')))

          se2 = se2.addBands(VVVH.rename('VVVH'))

          rgb = ['VV', 'VH', 'VVVH']


          url = se2.select(rgb).clip(aoi).getThumbURL({'min': -50, 'max': 1, 'region': aoi, 'format': 'jpg'})

          #add the name to my list I created earlier
          filelocations.append(filename)

          #now I open the url and download the image to the specified file location
          response = requests.get(url, stream=True)
          with open(filename, 'wb') as out_file:
            shutil.copyfileobj(response.raw, out_file)
          del response
        
        except Exception as e:
          filelocations.append("NULL")
        
      else:
        x+=1
        #print("skip")
        filelocations.append("NULL")

      if x % 100 == 0:
        print(x, "files complete")

  traindf['sentinel1_filelocation'] = filelocations

###Sentinel 1 Call

When both sentinel satellites are working you get a 6 day repeat. With one down it is a 12 day.

This calls a really long time range of 80 days, but selects the most proximal to the date of the SWE and returns the image for that only. 

In [16]:
##Test statement for ee API
try:
        ee.Initialize()
except Exception as e:
        ee.Authenticate()
        ee.Initialize()

pull_Sentinel(traindf, overwrite = True)



100 files complete
200 files complete
300 files complete
400 files complete
500 files complete
600 files complete
700 files complete
800 files complete
900 files complete
1000 files complete
1100 files complete
1200 files complete
1300 files complete
1400 files complete
1500 files complete
1600 files complete
1700 files complete
1800 files complete
1900 files complete
2000 files complete
2100 files complete
2200 files complete
2300 files complete
2400 files complete
2500 files complete
2600 files complete
2700 files complete
2800 files complete
2900 files complete
3000 files complete
3100 files complete
3200 files complete
3300 files complete
3400 files complete
3500 files complete
3600 files complete
3700 files complete
3800 files complete
3900 files complete
4000 files complete
4100 files complete
4200 files complete
4300 files complete
4400 files complete
4500 files complete
4600 files complete
4700 files complete
4800 files complete
4900 files complete
5000 files complete
5100 file

39700 files complete
39800 files complete
39900 files complete
40000 files complete
40100 files complete
40200 files complete
40300 files complete
40400 files complete
40500 files complete
40600 files complete
40700 files complete
40800 files complete
40900 files complete
41000 files complete
41100 files complete
41200 files complete
41300 files complete
41400 files complete
41500 files complete
41600 files complete
41700 files complete
41800 files complete
41900 files complete
42000 files complete
42100 files complete
42200 files complete
42300 files complete
42400 files complete
42500 files complete
42600 files complete
42700 files complete
42800 files complete
42900 files complete
43000 files complete
43100 files complete
43200 files complete
43300 files complete
43400 files complete
43500 files complete
43600 files complete
43700 files complete
43800 files complete
43900 files complete
44000 files complete
44100 files complete
44200 files complete
44300 files complete
44400 files c

78800 files complete
78900 files complete
79000 files complete
79100 files complete
79200 files complete
79300 files complete
79400 files complete
79500 files complete
79600 files complete
79700 files complete
79800 files complete
79900 files complete
80000 files complete
80100 files complete
80200 files complete
80300 files complete
80400 files complete
80500 files complete
80600 files complete
80700 files complete
80800 files complete
80900 files complete
81000 files complete
81100 files complete
81200 files complete
81300 files complete
81400 files complete
81500 files complete
81600 files complete
81700 files complete
81800 files complete
81900 files complete
82000 files complete
82100 files complete
82200 files complete
82300 files complete
82400 files complete
82500 files complete
82600 files complete
82700 files complete
82800 files complete
82900 files complete
83000 files complete
83100 files complete
83200 files complete
83300 files complete
83400 files complete
83500 files c

In [32]:
#Check to confirm file locations were written to dataframe
traindf

,cell_id,date,SWE,region,geometry,mean_inversed_swe,mean_local_swe,median_local_swe,max_local_swe,min_local_swe,...,center_long,MOD10A1_SnowCover,MOD10A1_Albedo,MOD10A1_NDSI,MYD10A1_SnowCover,MYD10A1_Albedo,MYD10A1_NDSI,sentinel1_filelocation,sentinel2a_filelocation,sentinel2b_filelocation
0,ASO_50M_SWE_USCARC_8,2017-07-17,48.561301,sierras,"POLYGON ((-119.27063 37.74391, -119.27063 37.7...",4.697841,4.685963,1.381988,18.482857,0.573913,...,-119.264960,0.761684,0.367174,0.766883,0.814966,0.345533,0.818492,/home/ubuntu/SnowData/Sen1_Data_poly/ASO_50M_S...,/home/ubuntu/SnowData/Sen2_DataA_poly/ASO_50M_...,/home/ubuntu/SnowData/Sen2_DataB_poly/ASO_50M_...
1,ASO_50M_SWE_USCARC_9,2017-07-17,33.924218,sierras,"POLYGON ((-119.27036 37.73490, -119.27036 37.7...",4.736198,4.685963,1.381988,18.482857,0.573913,...,-119.264686,0.666698,0.326389,0.669003,0.790095,0.339398,0.792235,/home/ubuntu/SnowData/Sen1_Data_poly/ASO_50M_S...,/home/ubuntu/SnowData/Sen2_DataA_poly/ASO_50M_...,/home/ubuntu/SnowData/Sen2_DataB_poly/ASO_50M_...
2,ASO_50M_SWE_USCARC_26,2017-07-17,31.831418,sierras,"POLYGON ((-119.25984 37.76214, -119.25984 37.7...",5.221427,4.685963,1.381988,18.482857,0.573913,...,-119.254165,0.717794,0.387605,0.721743,0.660893,0.377931,0.665706,/home/ubuntu/SnowData/Sen1_Data_poly/ASO_50M_S...,/home/ubuntu/SnowData/Sen2_DataA_poly/ASO_50M_...,/home/ubuntu/SnowData/Sen2_DataB_poly/ASO_50M_...
3,ASO_50M_SWE_USCARC_27,2017-07-17,41.723133,sierras,"POLYGON ((-119.25957 37.75313, -119.25957 37.7...",5.322858,4.685963,1.381988,18.482857,0.573913,...,-119.253892,0.758141,0.388365,0.761231,0.754647,0.370129,0.758559,/home/ubuntu/SnowData/Sen1_Data_poly/ASO_50M_S...,/home/ubuntu/SnowData/Sen2_DataA_poly/ASO_50M_...,/home/ubuntu/SnowData/Sen2_DataB_poly/ASO_50M_...
4,ASO_50M_SWE_USCARC_28,2017-07-17,42.693821,sierras,"POLYGON ((-119.25929 37.74413, -119.25929 37.7...",5.377803,4.685963,1.381988,18.482857,0.573913,...,-119.253619,0.732553,0.393211,0.737283,0.753537,0.346098,0.757798,/home/ubuntu/SnowData/Sen1_Data_poly/ASO_50M_S...,/home/ubuntu/SnowData/Sen2_DataA_poly/ASO_50M_...,/home/ubuntu/SnowData/Sen2_DataB_poly/ASO_50M_...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97636,ASO_50M_SWE_USCALB_79,2018-06-01,0.474991,sierras,"POLYGON ((-118.99236 37.58734, -118.99236 37.5...",2.126230,1.906877,1.888571,3.574384,0.000000,...,-118.986703,0.050713,0.056434,0.044083,0.018015,0.023185,-0.121343,/home/ubuntu/SnowData/Sen1_Data_poly/ASO_50M_S...,/home/ubuntu/SnowData/Sen2_DataA_poly/ASO_50M_...,/home/ubuntu/SnowData/Sen2_DataB_poly/ASO_50M_...
97637,ASO_50M_SWE_USCALB_87,2018-06-01,1.828016,sierras,"POLYGON ((-118.98152 37.60555, -118.98152 37.5...",2.115175,1.906877,1.888571,3.574384,0.000000,...,-118.975860,0.028636,0.053653,-0.007782,0.000000,0.000000,-0.069880,/home/ubuntu/SnowData/Sen1_Data_poly/ASO_50M_S...,/home/ubuntu/SnowData/Sen2_DataA_poly/ASO_50M_...,/home/ubuntu/SnowData/Sen2_DataB_poly/ASO_50M_...
97638,ASO_50M_SWE_USCALB_88,2018-06-01,0.114890,sierras,"POLYGON ((-118.98128 37.59654, -118.98128 37.5...",2.125414,1.906877,1.888571,3.574384,0.000000,...,-118.975621,0.143382,0.132695,0.066765,0.009245,0.012961,-0.126452,/home/ubuntu/SnowData/Sen1_Data_poly/ASO_50M_S...,/home/ubuntu/SnowData/Sen2_DataA_poly/ASO_50M_...,/home/ubuntu/SnowData/Sen2_DataB_poly/ASO_50M_...
97639,ASO_50M_SWE_USCALB_89,2018-06-01,2.170014,sierras,"POLYGON ((-118.98104 37.58753, -118.98104 37.5...",2.135945,1.906877,1.888571,3.574384,0.000000,...,-118.975383,0.115417,0.143560,0.091611,0.010301,0.015503,-0.018298,/home/ubuntu/SnowData/Sen1_Data_poly/ASO_50M_S...,/home/ubuntu/SnowData/Sen2_DataA_poly/ASO_50M_...,/home/ubuntu/SnowData/Sen2_DataB_poly/ASO_50M_...


#Sentinel 2 Function Bands B8A, B11, B12

In [18]:
def pull_Sentinel2a(traindf, overwrite = False, names_only = False ):
  ##Main loop that iterates over areas and stores images in file
  ##For test image of one AOI see below
  filelocations = []
  x= 0


  for i in range(len(traindf.SWE)):
    #create a name for the image
    pict_name = traindf.cell_id[i] + '_sentinel2a_poly' + traindf.datestring[i] + '.jpg'

    #create the whole filename with path to the correct folder
    filename = os.path.join('/home/ubuntu/SnowData/Sen2_DataA_poly', pict_name)

    if names_only:
      filelocations.append(filename)
      x += 1
      if x % 100 == 0:
        print(f'{x} files already exist')

    elif os.path.exists(filename) and not overwrite:
      filelocations.append(filename)
      x += 1
      if x % 100 == 0:
        print(f'{x} files already exist')

    else:

      #define area of interest by coordinates
      aoi = ee.Geometry.Polygon(list(traindf.geometry[i].exterior.coords))
      #print(aoi)
      start_date = traindf.date[i] - timedelta(days = 80)
      end_date = traindf.date[i] + timedelta(days = 1)

      year = start_date.year
      #print(year)

      if year >=2016:
        x+=1
        try:
          #print("calculating")
          # Sentinel-1 image filtered on date range and on aoi
          se2 = ee.ImageCollection('COPERNICUS/S2')\
            .filter(ee.Filter.date(ee.Date('2020-01-01'), ee.Date('2021-01-01')))\
            .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', 10))\
            .filterBounds(aoi) \
            .select(['B8A', 'B11', 'B12'])\
            .sort('system:time_start', False)\
            .first()

          #Create a ratio band of (B8a-B11)/(B8a+B11)

          BRatio = se2.expression(
              '((B8 - B11)/(B8 + B11))*100', {
                  'B8': se2.select('B8A'),
                  'B11': se2.select('B11')
              })

          se2 = se2.addBands(BRatio.rename('BRatio'))


          rgb = ['BRatio', 'B11', 'B12']


          url = se2.select(rgb).clip(aoi).getThumbURL({'min': -500, 'max':500, 'region': aoi, 'format': 'jpg'})

          #add the name to my list I created earlier
          filelocations.append(filename)

          #now I open the url and download the image to the specified file location
          response = requests.get(url, stream=True)
          with open(filename, 'wb') as out_file:
            shutil.copyfileobj(response.raw, out_file)
          del response
        
        except Exception as e:
          filelocations.append("NULL")
        
      else:
        x+=1
        #print("skip")
        filelocations.append("NULL")

      if x % 100 == 0:
        print(x, "files complete")

  traindf['sentinel2a_filelocation'] = filelocations

In [19]:
##Test statement for ee API
try:
        ee.Initialize()
except Exception as e:
        ee.Authenticate()
        ee.Initialize()

pull_Sentinel2a(traindf, overwrite = True)

100 files complete
200 files complete
300 files complete
400 files complete
500 files complete
600 files complete
700 files complete
800 files complete
900 files complete
1000 files complete
1100 files complete
1200 files complete
1300 files complete
1400 files complete
1500 files complete
1600 files complete
1700 files complete
1800 files complete
1900 files complete
2000 files complete
2100 files complete
2200 files complete
2300 files complete
2400 files complete
2500 files complete
2600 files complete
2700 files complete
2800 files complete
2900 files complete
3000 files complete
3100 files complete
3200 files complete
3300 files complete
3400 files complete
3500 files complete
3600 files complete
3700 files complete
3800 files complete
3900 files complete
4000 files complete
4100 files complete
4200 files complete
4300 files complete
4400 files complete
4500 files complete
4600 files complete
4700 files complete
4800 files complete
4900 files complete
5000 files complete
5100 file

39700 files complete
39800 files complete
39900 files complete
40000 files complete
40100 files complete
40200 files complete
40300 files complete
40400 files complete
40500 files complete
40600 files complete
40700 files complete
40800 files complete
40900 files complete
41000 files complete
41100 files complete
41200 files complete
41300 files complete
41400 files complete
41500 files complete
41600 files complete
41700 files complete
41800 files complete
41900 files complete
42000 files complete
42100 files complete
42200 files complete
42300 files complete
42400 files complete
42500 files complete
42600 files complete
42700 files complete
42800 files complete
42900 files complete
43000 files complete
43100 files complete
43200 files complete
43300 files complete
43400 files complete
43500 files complete
43600 files complete
43700 files complete
43800 files complete
43900 files complete
44000 files complete
44100 files complete
44200 files complete
44300 files complete
44400 files c

78800 files complete
78900 files complete
79000 files complete
79100 files complete
79200 files complete
79300 files complete
79400 files complete
79500 files complete
79600 files complete
79700 files complete
79800 files complete
79900 files complete
80000 files complete
80100 files complete
80200 files complete
80300 files complete
80400 files complete
80500 files complete
80600 files complete
80700 files complete
80800 files complete
80900 files complete
81000 files complete
81100 files complete
81200 files complete
81300 files complete
81400 files complete
81500 files complete
81600 files complete
81700 files complete
81800 files complete
81900 files complete
82000 files complete
82100 files complete
82200 files complete
82300 files complete
82400 files complete
82500 files complete
82600 files complete
82700 files complete
82800 files complete
82900 files complete
83000 files complete
83100 files complete
83200 files complete
83300 files complete
83400 files complete
83500 files c

#Sentinel 2 Function Bands B8 B4 B2

In [20]:
def pull_Sentinel2b(traindf, overwrite = False, names_only = False ):
  ##Main loop that iterates over areas and stores images in file
  ##For test image of one AOI see below
  filelocations = []
  x= 0


  for i in range(len(traindf.SWE)):
    #create a name for the image
    pict_name = traindf.cell_id[i] + '_sentinel2b_poly' + traindf.datestring[i] + '.jpg'

    #create the whole filename with path to the correct folder
    filename = os.path.join('/home/ubuntu/SnowData/Sen2_DataB_poly', pict_name)

    if names_only:
      filelocations.append(filename)
      x += 1
      if x % 100 == 0:
        print(f'{x} files already exist')

    elif os.path.exists(filename) and not overwrite:
      filelocations.append(filename)
      x += 1
      if x % 100 == 0:
        print(f'{x} files already exist')

    else:

      #define area of interest by coordinates
      aoi = ee.Geometry.Polygon(list(traindf.geometry[i].exterior.coords))
      #print(aoi)
      start_date = traindf.date[i] - timedelta(days = 80)
      end_date = traindf.date[i] + timedelta(days = 1)

      year = start_date.year
      #print(year)

      if year >=2016:
        x+=1
        try:
          #print("calculating")
          # Sentinel-1 image filtered on date range and on aoi
          se2 = ee.ImageCollection('COPERNICUS/S2')\
            .filter(ee.Filter.date(ee.Date('2020-01-01'), ee.Date('2021-01-01')))\
            .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', 10))\
            .filterBounds(aoi) \
            .select(['B8', 'B4', 'B2'])\
            .sort('system:time_start', False)\
            .first()

          #Create a ratio band of (B8a-B11)/(B8a+B11)

          BRatio = se2.expression(
              '((B8 - B4)/(B8 + B4))*10000', {
                  'B8': se2.select('B8'),
                  'B4': se2.select('B4')
              })

          se2 = se2.addBands(BRatio.rename('BRatio'))


          rgb = ['BRatio', 'B2', 'B4']


          url = se2.select(rgb).clip(aoi).getThumbURL({'min': -100, 'max':5000, 'region': aoi, 'format': 'jpg'})

          #add the name to my list I created earlier
          filelocations.append(filename)

          #now I open the url and download the image to the specified file location
          response = requests.get(url, stream=True)
          with open(filename, 'wb') as out_file:
            shutil.copyfileobj(response.raw, out_file)
          del response
        
        except Exception as e:
          filelocations.append("NULL")
        
      else:
        x+=1
        #print("skip")
        filelocations.append("NULL")

      if x % 100 == 0:
        print(x, "files complete")

  traindf['sentinel2b_filelocation'] = filelocations

In [39]:
lastbit = traindf.loc[80000:].reset_index(drop=True)

In [ ]:
pull_Sentinel2b(lastbit, overwrite = False)

100 files already exist
200 files already exist
300 files already exist
400 files already exist
500 files already exist
600 files already exist
700 files already exist
800 files already exist
900 files already exist
1000 files already exist
1100 files complete
1200 files complete
1300 files complete
1400 files complete
1500 files complete
1600 files complete
1700 files complete
1800 files complete
1900 files complete
2000 files complete
2100 files complete
2200 files complete
2300 files complete
2400 files complete
2500 files complete
2600 files complete
2700 files complete
2800 files complete
2900 files complete
3000 files complete
3100 files complete
3200 files complete
3300 files complete
3400 files complete
3500 files complete
3600 files complete
3700 files complete
3800 files complete
3900 files complete
4000 files complete
4100 files complete
4200 files complete
4300 files complete
4400 files complete
4500 files complete
4600 files complete
4700 files complete
4800 files complete

In [35]:
final = traindf[['cell_id', 'date', 'SWE', 'mean_inversed_swe',
       'mean_local_swe', 'median_local_swe', 'max_local_swe', 'min_local_swe',
       'mean_local_elevation', 'median_local_elevation', 'max_local_elevation',
       'min_local_elevation', 'MOD10A1_SnowCover', 'MOD10A1_Albedo', 'MOD10A1_NDSI', 
       'MYD10A1_SnowCover', 'MYD10A1_Albedo', 'MYD10A1_NDSI', 
       'copernicus_filelocations', 'sentinel1_filelocation', 'sentinel2a_filelocation',
       'sentinel2b_filelocation']]

In [36]:
#Write to csv 
final.to_csv('/home/ubuntu/SnowData/Final_CNN_Dataframe.csv', index=False)

#Sandbox Area

### Sentinel 1 Optimization SandBox

In [ ]:

import shapely.geometry

area_of_interest = traindf['geometry'][0]
shapely_polygon = shapely.geometry.Polygon(area_of_interest)
item = gpd.GeoSeries([shapely_polygon]).to_json()

item

In [ ]:
print(list(traindf.geometry[0].exterior.coords))

In [ ]:
pict_name = traindf.cell_id[0] + '_TESTSen2at_' + traindf.datestring[0] + '.jpg'


#create the whole filename with path to the correct folder
filename = os.path.join('/content/drive/MyDrive/snowcapstone team spring 2022/Sen_TEST', pict_name)


#define area of interest by coordinates
aoi = ee.Geometry.Polygon(list(traindf.geometry[0].exterior.coords))
      #print(aoi)
start_date = traindf.date[0] - timedelta(days = 800)
end_date = traindf.date[0] + timedelta(days = 1)

# Sentinel-1 image filtered on date range and on aoi
se2 = ee.ImageCollection('COPERNICUS/S2')\
.filter(ee.Filter.date(ee.Date('2020-01-01'), ee.Date('2021-01-01')))\
.filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', 10))\
.filterBounds(aoi) \
.select(['B8', 'B4', 'B2'])\
.sort('system:time_start', False)\
.first()


BRatio = se2.expression(
  '((B8 - B4)/(B8 + B4))*10000', {
  'B8': se2.select('B8'),
  'B4': se2.select('B4')
  })

se2 = se2.addBands(BRatio.rename('BRatio'))

rgb = ['BRatio', 'B2', 'B4']

##date = ee.Date(se2.get('system:time_start')).format("yyyy-MM-dd")
#print(date.getInfo())
url = se2.select(rgb).clip(aoi).getThumbURL({'min': -100, 'max':10000, 'region': aoi, 'format': 'jpg'})


#now I open the url and download the image to the specified file location
response = requests.get(url, stream=True)
with open(filename, 'wb') as out_file:
  shutil.copyfileobj(response.raw, out_file)
del response
        



In [ ]:
### Check out image
test1=plt.imread('/content/drive/MyDrive/snowcapstone team spring 2022/Sen_TEST/018cf1a1-f945-4097-9c47-0c4690538bb5_TESTSen2at_2016005.jpg')
print(test1.shape)
print(f"The image consists of {test1.shape[0] * test1.shape[1]} pixels")
plt.imshow(test1);

In [ ]:
test1

## Sentinel 2 ingestion

### Google Earth Engine version (not permissible for competition)

In [ ]:

def pull_Sentinel(traindf, overwrite = False, names_only = False ):
  ##Main loop that iterates over areas and stores images in file
  ##For test image of one AOI see below
  filelocations = []
  x= 0
  proj = ee.Projection('EPSG:32631')

  for i in range(len(traindf.SWE)):
    #create a name for the image
    pict_name = traindf.cell_id[i] + '_sentinel2_' + traindf.datestring[i] + '.jpg'

    #create the whole filename with path to the correct folder
    filename = os.path.join('/content/drive/MyDrive/snowcapstone team spring 2022/Sen2_Data',pict_name)

    if names_only:
      filelocations.append(filename)
      x += 1
      if x % 5000 == 0:
        print(f'{x} files already exist')

    elif os.path.exists(filename) and not overwrite:
      filelocations.append(filename)
      x += 1
      if x % 5000 == 0:
        print(f'{x} files already exist')

    else:

       #define area of interest by coordinates
      aoi = ee.Geometry.Point((traindf.center_long[i], traindf.center_lat[i]), proj = proj)
      #print(aoi)
      start_date = traindf.date[i] - timedelta(days = 90)
      end_date = traindf.date[i] + timedelta(days = 1)

      year = traindf.date[i].year
      #print(year)

          #umbImages = ee.ImageCollection('COPERNICUS/S2').filterDate(start_date, end_date).filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)).filterBounds(aoi).size();
          #print('numbImages', numbImages)
      if year >=2016:
        x+=1
        #print("calculating")
        rgb = ['B2','B11','B12']
        # Sentinel-2 image filtered on date range and on aoi
        se2 = ee.ImageCollection('COPERNICUS/S2') \
                        .filter(ee.Filter.date(start_date, end_date)) \
                        .filter(ee.Filter.notNull(['system:index'])) \
                        .select(rgb) \
                        .sort('system:index', False) 


        #Because the image collection is limited to a single day, there is only one image
        #So I just take it
        point = se2.first().reproject(crs = 'EPSG:32631').clip(aoi.buffer(100))

        # Get individual band arrays and build them into an RGB image
        # The "buffer" is a circular distance around the point, measured in meters right now it is 100km
        #rgb = ee.Image.rgb(point.clip(aoi.buffer(1)).select(rgb[0]), #I divide by 100 to get it between 0 and 1
        #                  point.clip(aoi.buffer(1)).select(rgb[1]), #I divide by 100 to get it between 0 and 1
        #                  point.clip(aoi.buffer(1)).select(rgb[2])).visualize() #I divide by 10000 to get it between 0 and 1

        #Now I get the url for the image
        url = point.getThumbURL({'bands': rgb, 'min': 0, 'max': 2500, 'gamma': 1.1})

        #add the name to my list I created earlier
        filelocations.append(filename)

        #now I open the url and download the image to the specified file location
        response = requests.get(url, stream=True)
        with open(filename, 'wb') as out_file:
          shutil.copyfileobj(response.raw, out_file)
        del response
        
      else:
        x+=1
        #print("skip")
        filelocations.append("No Data")

      if x % 5000 == 0:
        print(x, "files complete")

  traindf['sentinel2_filelocation'] = filelocations

In [ ]:
import IPython.display as disp
filelocations = []
x= 0
proj = ee.Projection('EPSG:32631')
for i in range(400,401):
  #define area of interest by coordinates
  aoi = ee.Geometry.Point((traindf.center_long[i], traindf.center_lat[i]), proj = proj)
  #print(aoi)
  start_date = traindf.date[i] - timedelta(days = 90)
  end_date = traindf.date[i] + timedelta(days = 1)

  year = traindf.date[i].year
  #print(year)

      #umbImages = ee.ImageCollection('COPERNICUS/S2').filterDate(start_date, end_date).filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)).filterBounds(aoi).size();
      #print('numbImages', numbImages)
  if year >=2016:
    x+=1
    #print("calculating")
    rgb = ['B2','B11','B12']
    # Sentinel-2 image filtered on date range and on aoi
    se2 = ee.ImageCollection('COPERNICUS/S2') \
                    .filter(ee.Filter.date(start_date, end_date)) \
                    .filter(ee.Filter.notNull(['system:index'])) \
                    .select(rgb) \
                    .sort('system:index', False) 


    #Because the image collection is limited to a single day, there is only one image
    #So I just take it
    point = se2.first().reproject(crs = 'EPSG:32631').clip(aoi.buffer(100))

    # Get individual band arrays and build them into an RGB image
    # The "buffer" is a circular distance around the point, measured in meters right now it is 100km
    #rgb = ee.Image.rgb(point.clip(aoi.buffer(1)).select(rgb[0]), #I divide by 100 to get it between 0 and 1
    #                  point.clip(aoi.buffer(1)).select(rgb[1]), #I divide by 100 to get it between 0 and 1
    #                  point.clip(aoi.buffer(1)).select(rgb[2])).visualize() #I divide by 10000 to get it between 0 and 1

    #Now I get the url for the image
    url = point.getThumbURL({'bands': rgb, 'min': 0, 'max': 2500, 'gamma': 1.1})

In [ ]:
url

In [ ]:
##Test statement for ee API
try:
        ee.Initialize()
except Exception as e:
        ee.Authenticate()
        ee.Initialize()

## Ingest data
#pull_Sentinel(traindf, overwrite = True, names_only= False)

Possible better image save solution: https://geemap.org/notebooks/21_export_map_to_html_png/

In [ ]:
test1=plt.imread('/content/drive/MyDrive/snowcapstone team spring 2022/Sen1_Data/2f5c1968-2c34-4539-a9f4-38298d0de925_sentinel1_VV_2016033.jpg')
print(test1.shape)
print(f"The image consists of {test1.shape[0] * test1.shape[1]} pixels")
plt.imshow(test1);

In [ ]:
test1

In [ ]:
test2

In [ ]:
test2=plt.imread('/content/drive/MyDrive/snowcapstone team spring 2022/Sen1_Data/2f266b81-aad3-4c26-ac69-1bd04a208bf7_sentinel1_VV_2016033.jpg')
print(test2.shape)
print(f"The image consists of {test2.shape[0] * test2.shape[1]} pixels")
plt.imshow(test2);


##Single trial example
### Sentinel 2 with cloud masking
https://developers.google.com/earth-engine/tutorials/community/sentinel-2-s2cloudless



In [ ]:
try:
        ee.Initialize()
except Exception as e:
        ee.Authenticate()
        ee.Initialize()

# get a single polygon
aoi = ee.Geometry.Polygon(traindf.geometry[0])
print(aoi)
#print(aoi)

# Sentinel-2 image filtered on 2019 and on aoi
se2 = ee.ImageCollection('COPERNICUS/S2').filterDate("2019-01-01","2019-12-31").filterBounds(aoi).median().divide(10000)

rgb = ['B12','B11','B2']

# set some thresholds
rgbViz = {"min":0.0, "max":0.9,"bands":rgb}


# initialize our map
map1 = geemap.Map()
map1.centerObject(aoi, 7)
map1.addLayer(se2.clip(aoi), rgbViz, "S2")

map1.addLayerControl()
map1

In [ ]:
##Cloud mask filster

def se2mask(image):
    quality_band = image.select('QA60')
    
    # using the bit mask for clouds and cirrus clouds respectively
    cloudmask = 1 << 10
    cirrusmask = 1 << 11
    
    # we only want clear skies
    mask = quality_band.bitwiseAnd(cloudmask).eq(0) and (quality_band.bitwiseAnd(cirrusmask).eq(0))
    
    # we'll divide by 10000 to make interpreting the reflectance values easier
    return image.updateMask(mask).divide(10000)
    
se2 = ee.ImageCollection('COPERNICUS/S2').filterDate(
    "2019-01-01","2019-12-31").filterBounds(aoi).filter(
    ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE",20)).map(se2mask).median()

# initialize our map
map2 = geemap.Map()
map2.centerObject(aoi, 7)
map2.addLayer(se2.clip(aoi), rgbViz, "S2")

map2.addLayerControl()
map2

In [ ]:
#se2.getInfo()

#url = se2.select(rgb).clip(aoi).getThumbURL({'min': -20, 'max': 0})

url

# Test Area

In [ ]:
pip install geemap

## GEE UTILS

In [ ]:
pip install geeutils

In [ ]:
import ee
import geemap

try:
        ee.Initialize()
except Exception as e:
        ee.Authenticate()
        ee.Initialize()
        
import sys
sys.path.append('../src/geepatches')
import geeutils

Testing with one 1k x 1k location in our region:

In [ ]:
aoi = ee.Geometry.Point((-121.939413466388, 41.1666473902759))
  #.buffer(100)

eeimagecollection = (ee.ImageCollection('COPERNICUS/S1_GRD')
                     .filter(ee.Filter.date(ee.Date('2020-01-01'), ee.Date('2021-01-01')))
                     .filterBounds(aoi.buffer(1000000)))


size_all     = (eeimagecollection.size().getInfo())
size_none    = (eeimagecollection
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV').Not())
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH').Not())
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'HV').Not())
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'HH').Not())
                .size().getInfo())
size_VV_only = (eeimagecollection
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH').Not())
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'HV').Not())
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'HH').Not())
                .size().getInfo())
size_VH_only = (eeimagecollection
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV').Not())
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'HV').Not())
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'HH').Not())
                .size().getInfo())
size_HV_only = (eeimagecollection
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV').Not())
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH').Not())
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'HV'))
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'HH').Not())
                .size().getInfo())
size_HH_only = (eeimagecollection
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV').Not())
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH').Not())
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'HV').Not())
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'HH'))
                .size().getInfo())
size_VVVH_only = (eeimagecollection
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'HV').Not())
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'HH').Not())
                .size().getInfo())
size_HVHH_only = (eeimagecollection
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV').Not())
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH').Not())
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'HV'))
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'HH'))
                .size().getInfo())
size_VVVHHVHH_only = (eeimagecollection
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'HV'))
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'HH'))
                .size().getInfo())
                        

#print(geeutils.szimagecollectioninfo(eeimagecollection))
print("At buffer of 1000000:")
print(f"all        : {size_all:15d}")
print(f"none       : {size_none:15d}")
print(f"VV only    : {size_VV_only:15d}   {size_VV_only/size_all:5.1%}")
print(f"VH only    : {size_VH_only:15d}   {size_VH_only/size_all:5.1%}")
print(f"HV only    : {size_HV_only:15d}   {size_HV_only/size_all:5.1%}")
print(f"HH only    : {size_HH_only:15d}   {size_HH_only/size_all:5.1%}")
print(f"VV+VH  only: {size_VVVH_only:15d}   {size_VVVH_only/size_all:5.1%}")
print(f"HV+HH  only: {size_HVHH_only:15d}   {size_HVHH_only/size_all:5.1%}")
print(f"VV+VH+HV+HH: {size_VVVHHVHH_only:15d}   {size_VVVHHVHH_only/size_all:5.1%}")

In [ ]:
aoi = ee.Geometry.Point((-121.939413466388, 41.1666473902759))
  #.buffer(100)

eeimagecollection = (ee.ImageCollection('COPERNICUS/S1_GRD')
                     .filter(ee.Filter.date(ee.Date('2020-01-01'), ee.Date('2021-01-01')))
                     .filterBounds(aoi.buffer(100)))


size_all     = (eeimagecollection.size().getInfo())
size_none    = (eeimagecollection
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV').Not())
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH').Not())
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'HV').Not())
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'HH').Not())
                .size().getInfo())
size_VV_only = (eeimagecollection
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH').Not())
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'HV').Not())
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'HH').Not())
                .size().getInfo())
size_VH_only = (eeimagecollection
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV').Not())
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'HV').Not())
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'HH').Not())
                .size().getInfo())
size_HV_only = (eeimagecollection
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV').Not())
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH').Not())
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'HV'))
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'HH').Not())
                .size().getInfo())
size_HH_only = (eeimagecollection
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV').Not())
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH').Not())
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'HV').Not())
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'HH'))
                .size().getInfo())
size_VVVH_only = (eeimagecollection
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'HV').Not())
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'HH').Not())
                .size().getInfo())
size_HVHH_only = (eeimagecollection
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV').Not())
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH').Not())
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'HV'))
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'HH'))
                .size().getInfo())
size_VVVHHVHH_only = (eeimagecollection
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'HV'))
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'HH'))
                .size().getInfo())
                        

#print(geeutils.szimagecollectioninfo(eeimagecollection))
print("At buffer of 100:")
print(f"all        : {size_all:15d}")
print(f"none       : {size_none:15d}")
print(f"VV only    : {size_VV_only:15d}   {size_VV_only/size_all:5.1%}")
print(f"VH only    : {size_VH_only:15d}   {size_VH_only/size_all:5.1%}")
print(f"HV only    : {size_HV_only:15d}   {size_HV_only/size_all:5.1%}")
print(f"HH only    : {size_HH_only:15d}   {size_HH_only/size_all:5.1%}")
print(f"VV+VH  only: {size_VVVH_only:15d}   {size_VVVH_only/size_all:5.1%}")
print(f"HV+HH  only: {size_HVHH_only:15d}   {size_HVHH_only/size_all:5.1%}")
print(f"VV+VH+HV+HH: {size_VVVHHVHH_only:15d}   {size_VVVHHVHH_only/size_all:5.1%}")

In [ ]:
aoi = ee.Geometry.Point((-121.939413466388, 41.1666473902759))
  #.buffer(100)

eeimagecollection = (ee.ImageCollection('COPERNICUS/S1_GRD')
                     .filter(ee.Filter.date(ee.Date('2020-01-01'), ee.Date('2021-01-01')))
                     .filterBounds(aoi.buffer(1000)))


size_all     = (eeimagecollection.size().getInfo())
size_none    = (eeimagecollection
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV').Not())
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH').Not())
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'HV').Not())
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'HH').Not())
                .size().getInfo())
size_VV_only = (eeimagecollection
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH').Not())
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'HV').Not())
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'HH').Not())
                .size().getInfo())
size_VH_only = (eeimagecollection
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV').Not())
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'HV').Not())
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'HH').Not())
                .size().getInfo())
size_HV_only = (eeimagecollection
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV').Not())
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH').Not())
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'HV'))
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'HH').Not())
                .size().getInfo())
size_HH_only = (eeimagecollection
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV').Not())
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH').Not())
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'HV').Not())
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'HH'))
                .size().getInfo())
size_VVVH_only = (eeimagecollection
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'HV').Not())
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'HH').Not())
                .size().getInfo())
size_HVHH_only = (eeimagecollection
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV').Not())
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH').Not())
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'HV'))
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'HH'))
                .size().getInfo())
size_VVVHHVHH_only = (eeimagecollection
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'HV'))
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'HH'))
                .size().getInfo())
                        

#print(geeutils.szimagecollectioninfo(eeimagecollection))
print("At buffer of 1000:")
print(f"all        : {size_all:15d}")
print(f"none       : {size_none:15d}")
print(f"VV only    : {size_VV_only:15d}   {size_VV_only/size_all:5.1%}")
print(f"VH only    : {size_VH_only:15d}   {size_VH_only/size_all:5.1%}")
print(f"HV only    : {size_HV_only:15d}   {size_HV_only/size_all:5.1%}")
print(f"HH only    : {size_HH_only:15d}   {size_HH_only/size_all:5.1%}")
print(f"VV+VH  only: {size_VVVH_only:15d}   {size_VVVH_only/size_all:5.1%}")
print(f"HV+HH  only: {size_HVHH_only:15d}   {size_HVHH_only/size_all:5.1%}")
print(f"VV+VH+HV+HH: {size_VVVHHVHH_only:15d}   {size_VVVHHVHH_only/size_all:5.1%}")

In [ ]:
aoi = ee.Geometry.Point((-121.939413466388, 41.1666473902759))
  #.buffer(100)

eeimagecollection = (ee.ImageCollection('COPERNICUS/S1_GRD')
                     .filter(ee.Filter.date(ee.Date('2020-01-01'), ee.Date('2021-01-01')))
                     .filterBounds(aoi.buffer(10000)))


size_all     = (eeimagecollection.size().getInfo())
size_none    = (eeimagecollection
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV').Not())
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH').Not())
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'HV').Not())
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'HH').Not())
                .size().getInfo())
size_VV_only = (eeimagecollection
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH').Not())
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'HV').Not())
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'HH').Not())
                .size().getInfo())
size_VH_only = (eeimagecollection
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV').Not())
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'HV').Not())
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'HH').Not())
                .size().getInfo())
size_HV_only = (eeimagecollection
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV').Not())
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH').Not())
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'HV'))
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'HH').Not())
                .size().getInfo())
size_HH_only = (eeimagecollection
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV').Not())
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH').Not())
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'HV').Not())
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'HH'))
                .size().getInfo())
size_VVVH_only = (eeimagecollection
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'HV').Not())
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'HH').Not())
                .size().getInfo())
size_HVHH_only = (eeimagecollection
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV').Not())
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH').Not())
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'HV'))
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'HH'))
                .size().getInfo())
size_VVVHHVHH_only = (eeimagecollection
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'HV'))
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'HH'))
                .size().getInfo())
                        

#print(geeutils.szimagecollectioninfo(eeimagecollection))
print("At buffer of 10000:")
print(f"all        : {size_all:15d}")
print(f"none       : {size_none:15d}")
print(f"VV only    : {size_VV_only:15d}   {size_VV_only/size_all:5.1%}")
print(f"VH only    : {size_VH_only:15d}   {size_VH_only/size_all:5.1%}")
print(f"HV only    : {size_HV_only:15d}   {size_HV_only/size_all:5.1%}")
print(f"HH only    : {size_HH_only:15d}   {size_HH_only/size_all:5.1%}")
print(f"VV+VH  only: {size_VVVH_only:15d}   {size_VVVH_only/size_all:5.1%}")
print(f"HV+HH  only: {size_HVHH_only:15d}   {size_HVHH_only/size_all:5.1%}")
print(f"VV+VH+HV+HH: {size_VVVHHVHH_only:15d}   {size_VVVHHVHH_only/size_all:5.1%}")

In [ ]:
eeregion          = ee.Geometry.Point((-121.939413466388, 41.1666473902759)).buffer(1000)
#eeregion          = geeutils.squareareaboundsroi(eepoint, 1000) # 2km x 2km
eedatefrom        = ee.Date('2016-01-01')
eedatetill        = ee.Date('2021-01-01')            # Beware for the "User memory limit exceeded."

eeimagecollection = (ee.ImageCollection('COPERNICUS/S1_GRD')
                     .filterBounds(eeregion)
                     .filter(ee.Filter.date(eedatefrom, eedatetill)))

#
# collection info
#
#print(f"S1_GRD szimagecollectioninfo: \n{geeutils.szimagecollectioninfo(eeimagecollection)}\n")
#
# estimate ranges
#
size_all_nf     = (eeimagecollection.size().getInfo())
print(f"all (nf)        : {size_all:15d}")



eeimagecollectionf = (ee.ImageCollection('COPERNICUS/S1_GRD_FLOAT')
                     .filterBounds(eeregion)
                     .filter(ee.Filter.date(eedatefrom, eedatetill)))
#
# collection info
#
#print(f"S1_GRD_FLOAT szimagecollectioninfo: \n{geeutils.szimagecollectioninfo(eeimagecollection)}\n")
#
# estimate ranges
#
#print(f"S1_GRD_FLOAT szestimatevaluesinfo: \n{geeutils.szestimatevaluesinfo(eeimagecollection)}\n")

size_all_f     = (eeimagecollectionf.size().getInfo())
print(f"all (f)        : {size_all:15d}")



###################################################
# convert natural to decibel
###################################################
def tosigmadb(image):
    return (image
            .addBands(ee.Image(image.select('VV').log10().multiply(10.0)), overwrite=True)
            .addBands(ee.Image(image.select('VH').log10().multiply(10.0)), overwrite=True))

eeimagecollection = eeimagecollection.map(tosigmadb)

#
# collection info
#
#print(f"tosigmadb S1_GRD_FLOAT szimagecollectioninfo: \n{geeutils.szimagecollectioninfo(eeimagecollection)}\n")
#
# estimate ranges
#
size_convert    = (eeimagecollectionf.size().getInfo())

print(f"tosigmadb S1_GRD_FLOAT szestimatevaluesinfo: \n{geeutils.szestimatevaluesinfo(eeimagecollection)}\n")



eeimagecollection = (ee.ImageCollection('COPERNICUS/S1_GRD')
                     .filterBounds(eeregion)
                     .filter(ee.Filter.date(eedatefrom, eedatetill)))

###################################################
# convert decibel to natural
###################################################
def fromsigmadb(image):
    return (image
            .addBands(ee.Image(ee.Image(10.0).pow(image.select('VV').divide(10.0))).select([0], ['VV']), overwrite=True)
            .addBands(ee.Image(ee.Image(10.0).pow(image.select('VH').divide(10.0))).select([0], ['VH']), overwrite=True))

eeimagecollection = eeimagecollection.map(fromsigmadb)

#
# collection info
#
#print(f"fromsigmadb S1_GRD szimagecollectioninfo: \n{geeutils.szimagecollectioninfo(eeimagecollection)}\n")
#
# estimate ranges
#
print(f"fromsigmadb S1_GRD szestimatevaluesinfo: \n{geeutils.szestimatevaluesinfo(eeimagecollection)}\n")

###Sentinel 2

In [ ]:
aoi = ee.Geometry.Point((-121.939413466388, 41.1666473902759))
  #.buffer(100)

eeimagecollection = (ee.ImageCollection('COPERNICUS/S2')
                     .filter(ee.Filter.date(ee.Date('2020-01-01'), ee.Date('2021-01-01')))
                     .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', 20))
                     .filterBounds(aoi.buffer(10000))
                     .select(['B2', 'B11', 'B12']))


bands = (ee.ImageCollection('COPERNICUS/S2')
                     .filter(ee.Filter.date(ee.Date('2020-01-01'), ee.Date('2021-01-01')))
                     .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', 20))
                     .select(['B12'])
                     .filterBounds(aoi.buffer(10000)))

size_all     = (eeimagecollection.size().getInfo())
bands_size    = (bands.size().getInfo())
size_none    = (eeimagecollection
                .filter(ee.Filter.listContains('B2').Not())
                .filter(ee.Filter.listContains('B11').Not())
                .filter(ee.Filter.listContains('B12').Not())
                .size().getInfo())
size_B2_only = (eeimagecollection
                .filter(ee.Filter.listContains('B2'))
                .filter(ee.Filter.listContains('B11').Not())
                .filter(ee.Filter.listContains('B12').Not())
                .size().getInfo())
size_B11_only = (eeimagecollection
                .filter(ee.Filter.listContains('B2').Not())
                .filter(ee.Filter.listContains('B11'))
                .filter(ee.Filter.listContains('B12').Not())
                .size().getInfo())
size_B12_only = (eeimagecollection
                .filter(ee.Filter.listContains('B2').Not())
                .filter(ee.Filter.listContains('B11').Not())
                .filter(ee.Filter.listContains('B12'))
                .size().getInfo())

size_B2B11B12_only = (eeimagecollection
                .filter(ee.Filter.listContains('B2'))
                .filter(ee.Filter.listContains('B11'))
                .filter(ee.Filter.listContains('B12'))
                .size().getInfo())
size_B2B3B4_only = (eeimagecollection
                .filter(ee.Filter.listContains('B2'))
                .filter(ee.Filter.listContains('B3'))
                .filter(ee.Filter.listContains('B4'))
                .size().getInfo())

                        

#print(geeutils.szimagecollectioninfo(eeimagecollection))
print("At buffer of 100:")
print(f"all        : {size_all:15d}")
print(f"bands        : {bands_size:15d}")
print(f"none       : {size_none:15d}")
print(f"B2 only    : {size_B2_only:15d}   {size_B2_only/size_all:5.1%}")
print(f"B11 only    : {size_B11_only:15d}   {size_B11_only/size_all:5.1%}")
print(f"B12 only    : {size_B12_only:15d}   {size_B12_only/size_all:5.1%}")
print(f"B2+B11+B12  only: {size_B2B11B12_only:15d}   {size_B2B11B12_only/size_all:5.1%}")
print(f"B2+B3+B4  only: {size_B2B3B4_only:15d}   {size_B2B3B4_only/size_all:5.1%}")


In [ ]:
info = (eeimagecollection.getInfo())
print(info)

In [ ]:
info

In [ ]:
eeimagecollection1 = (ee.ImageCollection('COPERNICUS/S1_GRD')
                     .filter(ee.Filter.date(ee.Date('2020-01-01'), ee.Date('2021-01-01')))
                     .filterBounds(aoi.buffer(10000))
                    .filter(ee.Filter.listContains('transmitterReceiverPolarisation')))

In [ ]:
info1 =(eeimagecollection1.getInfo())

In [ ]:
info1